# Neutron Event Analyzer Tutorial

This tutorial demonstrates how to use the `neutron_event_analyzer` package to process and analyze neutron event camera data. The package processes `.empirphot` and `.empirevent` files, associates photons with events, computes ellipticity, and generates diagnostic plots. It supports two association methods: a simple method and an optional `lumacamTesting`-based method (if installed).

## Setup

Ensure you have installed the package:

```bash
pip install neutron_event_analyzer
```

For the `lumacamTesting` association method, install the optional dependency:

```bash
pip install neutron_event_analyzer[lumacam]
```

You need a data folder with the following structure:

```
data_folder/
├── photonFiles/
│   └── *.empirphot
└── eventFiles/
    └── *.empirevent
```

Additionally, provide the path to the `export` directory containing the `empir_export_events` and `empir_export_photons` binaries.

## Prerequisites

Let's import the necessary module and set up the environment.

In [ ]:
from neutron_event_analyzer import Analyse
import numpy as np

# Define paths (replace with your actual paths)
data_folder = '/path/to/your/data_folder'
export_dir = '/path/to/your/export'

## Step 1: Initialize the Analyser

Create an `Analyse` object, specifying the data folder, export directory, and whether to use `lumacamTesting` (if available).

In [ ]:
# Initialize with simple association
analyser_simple = Analyse(data_folder=data_folder, export_dir=export_dir, n_threads=4, use_lumacam=False)
print("Simple analyser initialized.")

# Initialize with lumacamTesting (if installed)
analyser_lumacam = Analyse(data_folder=data_folder, export_dir=export_dir, n_threads=4, use_lumacam=True)
print("Lumacam analyser initialized.")

## Step 2: Load Event and Photon Data

Load the event and photon files into DataFrames. You can specify custom glob patterns if needed.

In [ ]:
# Load events
analyser_simple.load_events()
analyser_lumacam.load_events()

# Load photons
analyser_simple.load_photons()
analyser_lumacam.load_photons()

## Step 3: Associate Photons to Events

Associate photons to events using either the simple method or the `lumacamTesting` method.

In [ ]:
# Simple association
analyser_simple.associate(time_norm_ns=1.0, spatial_norm_px=1.0, dSpace_px=np.inf, verbosity=1)

# Lumacam association (if lumacamTesting is installed)
analyser_lumacam.associate(weight_px_in_s=None, max_dist_s=None, verbosity=1)

## Step 4: Compute Ellipticity

Compute ellipticity and related metrics for multi-photon events. Use the appropriate event column for lumacam (`assoc_cluster_id`).

In [ ]:
analyser_simple.compute_ellipticity(verbosity=1)
analyser_lumacam.compute_ellipticity(event_col='assoc_cluster_id', verbosity=1)

## Step 5: Retrieve Combined DataFrame

Get the DataFrame containing associated photon and event data, including ellipticity metrics.

In [ ]:
combined_df_simple = analyser_simple.get_combined_dataframe()
combined_df_simple.head()

combined_df_lumacam = analyser_lumacam.get_combined_dataframe()
combined_df_lumacam.head()

## Step 6: Generate Diagnostic Plots

Create four diagnostic plots to visualize event data.

In [ ]:
analyser_simple.plot_four(name='MyRun_Simple', min_n=1, max_n=1000, min_psd=1e-10, max_psd=1)

## Step 7: Plot a Specific Event

Visualize a specific event to see its associated photons and event center.

In [ ]:
# Plot an event from the simple association
event_id = combined_df_simple['assoc_event_id'].dropna().iloc[0]  # First non-NaN event ID
analyser_simple.plot_event(event_id=event_id, title=f'Simple Association Event {event_id}')

# Plot an event from the lumacam association
event_id_lumacam = combined_df_lumacam['assoc_cluster_id'].dropna().iloc[0]  # First non-NaN cluster ID
analyser_lumacam.plot_event(event_id=event_id_lumacam, event_col='assoc_cluster_id', title=f'Lumacam Association Event {event_id_lumacam}')

## Notes

- Ensure the `export_dir` contains the required binaries.
- The `lumacamTesting` method requires the optional dependency and uses `assoc_cluster_id` instead of `assoc_event_id`.
- Adjust `n_threads` based on your system's capabilities.
- The `plot_four` method generates hexbin plots and a histogram for event analysis.
- The `plot_event` method visualizes individual events with their photons and event center.
- For large datasets, processing may take time; monitor progress via the `tqdm` bars.

For more details, refer to the [README](README.md) file in the package directory.